In [2]:
# %load_ext lab_black

# Standard Imports

In [3]:
import sys

# sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append("../build/mechsystem")

from mass_spring import *

from pythreejs import *

## Double Pendulum
### Mass definiton

In [4]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

mA = mss.add(Mass(1, (1, 0, 0)))
mB = mss.add(Mass(2, (2, 0, 0)))
f1 = mss.add(Fix((0, 0, 0)))
mss.add(Spring(1, 200000, (f1, mA)))
mss.add(Spring(1, 100000, (mA, mB)))

1

### Definition of the System

In [5]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(
            SphereBufferGeometry(0.2, 16, 16),
            MeshStandardMaterial(color="red"),
            position=m.pos,
        )
    )

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(
            SphereBufferGeometry(0.2, 32, 16),
            MeshStandardMaterial(color="blue"),
            position=f.pos,
        )
    )

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append([pA, pB])

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color="cyan")
springs = LineSegments2(springgeo, m2)

axes = AxesHelper(1)

### Definiton of the View

In [6]:
view_width = 600
view_height = 400

camera = PerspectiveCamera(
    position=[0, 10, 0],
    aspect=view_width / view_height,
    look_at=[0, 0, 0],  # look at origin
    up=[0, 0, 1],
)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(
    children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light]
)
controller = OrbitControls(controlling=camera)
renderer = Renderer(
    camera=camera,
    scene=scene,
    controls=[controller],
    width=view_width,
    height=view_height,
)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(0.0, 10.0, 0.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0,…

### Simulation of the System

In [7]:
from time import sleep

for i in range(10000):
    mss.simulate(0.02, 100)
    for m, mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append([pA, pB])
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)

KeyboardInterrupt: 

## Double Pendulum to chanin

In [11]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

f0 = mss.add(Fix((0, 0, 0)))  # Fixed anchor
m1 = mss.add(Mass(1.0, (0.0, 0.0, -1.0)))  # First mass
mss.add(Spring(1.0, 50000, (f0, m1)))  # First chain link
# Double pendulum
m2 = mss.add(Mass(1.0, (0.5, 0.0, -2.0)))
m3 = mss.add(Mass(1.0, (1.0, 0.0, -3.0)))
mss.add(Spring(1.0, 40000, (m1, m2)))
mss.add(Spring(1.0, 40000, (m2, m3)))

2

In [ ]:
mvis = []
for m in mss.masses:
    mvis.append(
        Mesh(
            SphereBufferGeometry(0.15, 16, 16),
            MeshStandardMaterial(color="red"),
            position=m.pos,
        )
    )

fvis = []
for f in mss.fixes:
    fvis.append(
        Mesh(
            SphereBufferGeometry(0.2, 16, 16),
            MeshStandardMaterial(color="blue"),
            position=f.pos,
        )
    )


# Build spring positions
def get_spring_segments():
    segs = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        segs.append([pA, pB])
    return segs


spring_geo = LineSegmentsGeometry(positions=get_spring_segments())
spring_mat = LineMaterial(linewidth=3, color="cyan")
springs = LineSegments2(spring_geo, spring_mat)

### Definiton of the View

In [53]:
view_width = 600
view_height = 400

axes = AxesHelper(1)
camera = PerspectiveCamera(
    pposition=[3, 6, 4], aspect=view_width / view_height, up=[0, 0, 1]
)
camera.lookAt([0, 0, -1])

scene = Scene(
    children=[
        *mvis,
        *fvis,
        springs,
        axes,
        camera,
        DirectionalLight(position=[0, 10, 10]),
        AmbientLight(),
    ]
)

controller = OrbitControls(controlling=camera)
renderer = Renderer(
    camera=camera,
    scene=scene,
    controls=[controller],
    width=view_width,
    height=view_height,
)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(0.0, 0.0, 0.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, …

In [52]:
for _ in range(20000):
    # real physics integration via Newmark (C++)
    mss.simulate(0.02, 100)

    # update mass meshes
    for m, vis in zip(mss.masses, mvis):
        vis.position = (m.pos[0], m.pos[1], m.pos[2])

    # update springs
    spring_geo = LineSegmentsGeometry(positions=get_spring_segments())
    springs.geometry = spring_geo

    sleep(0.01)

KeyboardInterrupt: 